In [11]:
import pandas as pd
import json
import re
import pymongo
import jieba
import operator
import numpy as np

In [2]:
#連接 mongodb

client = pymongo.MongoClient(host='123.241.175.34', port=27017)

In [3]:
#mongodb帳號密碼

client.admin.authenticate('root', '1qaz@WSX3edc')








True

In [4]:
#設定名稱為test的資料庫 也可用 db = client['test']
client.database_names()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: database_names is deprecated. Use list_database_names instead.
  


['ETL', 'admin', 'config', 'local', 'raw_article', 'test']

In [4]:
db_ETL=client.ETL

In [12]:
#顯示db中所有collections
db_ETL.list_collection_names()

['article_news_seg', 'article_web_seg', 'article_social_seg']

In [13]:
article_social=db_ETL.article_social_seg.find()
article_web=db_ETL.article_web_seg.find()
article_news=db_ETL.article_news_seg.find()

In [14]:
df_article_social=pd.DataFrame(list(article_social))
df_article_web=pd.DataFrame(list(article_web))
df_article_news=pd.DataFrame(list(article_news))

In [15]:
article_social_seg=df_article_social['seg']
article_web_seg=df_article_web['seg']
article_news_seg=df_article_news['seg']

In [16]:
article_news_seg[:5]

0    [國內, 刷卡, 回饋, 重, 洗牌, 　, 無腦, 刷, 排行榜, 大車, 拼, 　, 　...
1    [直擊雙, 11, 電商, AI, 倉儲, 　, 下, 單到, 出貨僅, 10, 分鐘, 　...
2    [雙, 11, 網購, 主打, 賓士車, 　, 交易, 攻略, 回饋, 16.8, 萬元, ...
3    [刷, 信用卡, 繳納地, 價稅, 　, 24, 家銀行, 免收, 手續費, 　, 　, 今...
4    [4, 張, 行動支付, 聯名卡, 對決, 　, 街口, 卡力尬, Pi, 拍錢包, 　, ...
Name: seg, dtype: object

In [17]:
bank={'中信':'中信','中國信託':'中信' ,'中信金':'中信' ,'中信金控':'中信','中信銀':'中信',
'玉山':'玉山','玉山金':'玉山','玉山金控':'玉山','玉山銀':'玉山',
'Richart':'Richart', 'richart':'Richart',
'KOKO':'KOKO','koko':'KOKO',
'國泰':'國泰','國泰世華':'國泰','國泰金':'國泰','國泰金控':'國泰',
'台新':'台新','台新金':'台新','台新金控':'台新','台新銀':'台新','台新銀行':'台新',
'永豐':'永豐','永豐銀':'永豐','永豐金':'永豐','永豐金控':'永豐','永豐銀行':'永豐',
'兆豐':'兆豐','兆豐銀':'兆豐','兆豐金':'兆豐','兆豐金控':'兆豐','兆豐銀行':'兆豐',
'花旗':'花旗','花旗銀':'花旗','花旗銀行':'花旗',
'一銀':'一銀','第一銀':'一銀','第一金':'一銀','第一金控':'一銀','第一銀行':'一銀',
'富邦':'富邦','富邦金':'富邦','富邦金控':'富邦','富邦銀':'富邦','富邦銀行':'富邦',
'彰化':'彰化','彰化銀':'彰化','彰化金控':'彰化','彰化金':'彰化','彰化銀行':'彰化',
'新光':'新光','新光銀':'新光','新光金':'新光','新光金控':'新光','新光銀行':'新光',
'匯豐':'匯豐','匯豐銀行':'匯豐','匯豐銀':'匯豐',
'陽信銀行':'陽信','陽信':'陽信','陽信銀':'陽信',
'元大':'元大','元大銀行':'元大','元大金控':'元大','元大銀':'元大',
'星展':'星展','星展銀行':'星展','星展銀':'星展','DBS':'星展','dbs':'星展',
'渣打':'渣打','渣打銀行':'渣打','渣打銀':'渣打',
'上海商銀':'上海商銀','上海銀行':'上海商銀','上海商業':'上海商銀',
'華南':'華南','華南銀行':'華南','華南金控':'華南','華南銀':'華南',
'凱基':'凱基','凱基銀行':'凱基','凱基金控':'凱基',
'合作金庫':'合作金庫','合庫':'合作金庫',
'王道':'王道','王道銀行':'王道',
'澳盛':'澳盛','澳盛銀行':'澳盛','ANZ':'澳盛','anz':'澳盛',
'遠東商銀':'遠東商銀','遠銀':'遠東商銀','遠東銀':'遠東商銀',
'土地銀行':'土地','土銀':'土地',
'聯邦銀行':'聯邦','聯邦':'聯邦',
'台灣銀行':'臺灣銀行','臺灣銀行':'臺灣銀行',
'京城':'京城','京城銀行':'京城',
'日盛':'日盛','日盛銀行':'日盛',
'華泰':'華泰','華泰銀行':'華泰',
'台中銀行':'台中銀行','台中銀':'台中銀行',
'高雄銀行':'高雄銀行','高雄銀':'高雄銀行',
'台灣企銀':'臺灣企銀','臺灣企銀':'臺灣企銀','臺企銀':'臺灣企銀',
'瑞興銀行':'瑞興銀行','瑞興':'瑞興銀行',
}

behavior={
'跑步':'運動','健身房':'運動','運動':'運動',
'旅遊':'旅遊','旅行':'旅遊','機票':'旅遊','飯店':'旅遊','機場':'旅遊','出國':'旅遊',
'保險':'保險','保費':'保險',
'加油':'加油','中油':'加油',
'國外':'國外','海外':'國外',
'電影':'電影','國賓':'電影','威秀':'電影',
'網購':'網購' ,'網拍':'網購', '蝦皮':'網購', '電商':'網購','pchome':'網購','PChome':'網購','Yahoo':'網購','momo':'網購','MOMO':'網購',
'行動支付':'行動支付','PI錢包':'行動支付','pi錢包':'行動支付','街口':'行動支付','Pi錢包':'行動支付',
'新光三越':'百貨','SOGO':'百貨','sogo':'百貨','遠百':'百貨','遠東百貨':'百貨',
'紅利點數':'紅利','紅利兌換':'紅利',
'超商':'超商','7-11':'超商','全家':'超商','全家便利':'超商','萊爾富':'超商','OK便利':'超商',
'屈臣氏':'藥妝','康是美':'藥妝',
'大潤發':'量販店','家樂福':'量販店','愛買':'量販店',
'全聯':'超市','頂好':'超市',
'哩程':'航空哩程','航空哩程':'航空哩程'
}


card={
'linepay':'LINEPay','Linepay':'LINEPay','LinePay':'LINEPay','LINEPAY':'LINEPay','LINE PAY':'LINEPay','Line pay':'LINEPay',
'line pay':'LINEPay','Line':'LINEPay','LINE':'LINEPay','line':'LINEPay',
'GOGO':'GoGo','gogo':'GoGo','黑狗':'GoGo','黑狗卡':'GoGo','gogo卡':'GoGo','GOGO卡':'GoGo','Gogo卡':'GoGo','Gogo':'GoGo','GoGo':'GoGo',
'flygo':'Flygo','Flygo':'Flygo','FLYGO':'Flygo','飛狗':'Flygo','飛狗卡':'Flygo','FlyGo':'Flygo','fly':'Flygo',
'大戶':'dawho','dawho':'dawho','Dawho':'dawho','DAWHO':'dawho',
'PI':'Pi','Pi':'Pi','pi卡':'Pi','PI卡':'Pi',
'ubear':'Ubear','UBEAR':'Ubear','Ubear':'Ubear','ubear':'Ubear','bear':'Ubear','BEAR':'Ubear',
'u bear':'Ubear','U bear':'Ubear','u bear':'Ubear', 'Ubear卡':'Ubear','U Bear':'Ubear',
'only':'Only','ONLY':'Only','Only':'Only',
'賴點':'賴點卡','賴點卡':'賴點卡', 
'酷玩卡':'酷玩卡','酷玩卡':'酷玩卡',
'饗樂':'饗樂卡','饗樂卡':'饗樂卡',
'koko':'KOKO','KOKO':'KOKO','KoKo':'KOKO','Koko':'KOKO',
'星燦':'星燦卡','星燦卡':'星燦卡','星璨':'星燦卡',
'統一獅':'統一獅聯名卡',
'中信兄弟':'中信兄弟聯名卡',
'富邦悍將':'富邦悍將聯名卡',
'My樂現金回饋卡':'My樂現金回饋卡','My樂現金回饋':'My樂現金回饋卡',
'哆啦A夢JCB聯名卡':'哆啦A夢JCB聯名卡','哆啦A夢':'哆啦A夢JCB聯名卡',
'Omiyage':'Omiyage','omiyage':'Omiyage',
'尊御世界卡':'尊御世界卡',
'鑽金一卡通聯名卡':'鑽金','鑽金':'鑽金',
'炫晶御璽卡':'炫晶御璽卡','炫晶卡':'炫晶御璽卡','炫晶':'炫晶御璽卡',
'現金回饋御璽卡':'現金回饋御璽卡',
'PChome Prime聯名卡':'PChome Prime聯名卡','Prime聯名卡':'PChome Prime聯名卡','Prime':'PChome Prime聯名卡','rime':'PChome Prime聯名卡',
'蝦皮購物聯名卡':'蝦皮購物聯名卡','蝦皮聯名':'蝦皮購物聯名卡',
'Lamigo聯名卡':'Lamigo聯名卡','Lamigo':'Lamigo聯名卡','lamigo':'Lamigo聯名卡',
'SPORT卡':'SPORT卡','SPORT':'SPORT卡',
'e秒刷鈦金卡':'e秒刷鈦金卡','e秒刷':'e秒刷鈦金卡',
'小小兵信用卡':'小小兵信用卡','小小兵':'小小兵信用卡',
'everyday鈦金卡':'everyday鈦金卡','everyday':'everyday鈦金卡',
'街口聯名卡':'街口聯名卡','街口聯名':'街口聯名卡',
'Gogoro聯名卡':'Gogoro聯名卡','Gogoro':'Gogoro聯名卡','gogoro':'Gogoro聯名卡',
'現金回饋御璽卡':'現金回饋御璽卡',
'簡單卡':'簡單卡',
'數位生活':'數位生活卡',
'美國運通':'美國運通','法拉利':'法拉利無限卡',
'omiyage':'Omiyage','OMIYAGE':'Omiyage','omiyage':'Omiyage',
'e秒刷鈦金卡':'e秒刷鈦金卡','e秒刷':'e秒刷鈦金卡',
'南開科技大學認同卡':'南開科技大學認同卡',
'聯邦OTA手機信用卡':'聯邦OTA手機信用卡',
'魔法少女iPASS一卡通聯名卡':'魔法少女iPASS一卡通聯名卡',
'富邦鑽保卡':'富邦鑽保卡',
'行遍天下聯名卡':'行遍天下聯名卡',
'幸運鈦金卡':'幸運鈦金卡',
'昇恒昌聯名卡':'昇恒昌聯名卡',
'新光三越一卡通聯名卡':'新光三越一卡通聯名卡',
'悠遊晶緻卡':'悠遊晶緻卡',
'鈦金商務卡':'鈦金商務卡',
'永豐Me Card':'永豐Me Card',
'永豐鈦豐卡':'永豐鈦豐卡',
'利倍卡':'利倍卡',
'富邦財神卡':'富邦財神卡',
'八福公益卡':'八福公益卡',
'台灣銀行':'台灣銀行',
'F1加油卡':'F1加油卡',
'全家Fish悠遊聯名卡':'全家Fish悠遊聯名卡',
'新光銀行寰宇卡':'新光銀行寰宇卡',
'建築師無限卡':'建築師無限卡',
'icash聯名卡':'icash聯名卡',
'世界卡(icash)':'世界卡(icash)',
'玉山教師卡':'玉山教師卡',
'安泰銀行信用卡':'安泰銀行信用卡',
'寰宇現金回饋卡':'寰宇現金回饋卡',
'合作金庫無限卡':'合作金庫無限卡',
'合庫雙幣信用卡(日圓)':'合庫雙幣信用卡(日圓)',
'一卡通聯名卡':'一卡通聯名卡',
'TAIPEI 101 聯名卡':'TAIPEI 101 聯名卡',
'公務人員國民旅遊卡':'公務人員國民旅遊卡',
'C’est Moi我的卡':'C’est Moi我的卡',
'Super Life 卡':'Super Life 卡',
'聯邦銀行無限卡':'聯邦銀行無限卡',
'匯豐銀行白金卡':'匯豐銀行白金卡',
'樂活Visapaywave感應式白金卡':'樂活Visapaywave感應式白金卡',
'臺北科大愛校認同卡':'臺北科大愛校認同卡',
'新光三越icash聯名卡':'新光三越icash聯名卡',
'鈦金悠遊聯名卡':'鈦金悠遊聯名卡',
'公司戶商務卡':'公司戶商務卡',
'樂享晶緻卡(原大眾)':'樂享晶緻卡(原大眾)',
'華歌爾聯名卡':'華歌爾聯名卡',
'高雄going鈦金卡':'高雄going鈦金卡',
'長庚紀念醫院悠遊認同卡':'長庚紀念醫院悠遊認同卡',
'樂購卡':'樂購卡',
'雙幣信用卡':'雙幣信用卡',
'鈦金商旅卡':'鈦金商旅卡',
'大台北商銀聯名卡':'大台北商銀聯名卡',
'KOKOicash聯名卡':'KOKOicash聯名卡',
'商務卡':'商務卡',
'商旅鈦金卡商務':'商旅鈦金卡商務',
'花蓮一信聯名卡':'花蓮一信聯名卡',
'元大白金卡':'元大白金卡',
'燦坤生活一卡通聯名卡':'燦坤生活一卡通聯名卡',
'圓山大飯店一卡通聯名卡':'圓山大飯店一卡通聯名卡',
'鹿港天后宮晶緻一卡通認同卡':'鹿港天后宮晶緻一卡通認同卡',
'台塑(一卡通)聯名卡':'台塑(一卡通)聯名卡',
'ANA聯名卡':'ANA聯名卡',
'日盛銀行-無限卡':'日盛銀行-無限卡',
'台中二信聯名卡':'台中二信聯名卡',
'一銀卡御守系列':'一銀卡御守系列',
'台灣國際基督教會認同卡':'台灣國際基督教會認同卡',
'NFC 手機信用卡':'NFC 手機信用卡',
'雅芳聯名卡':'雅芳聯名卡',
'哆啦A夢晶緻卡':'哆啦A夢晶緻卡',
'NFC手機信用卡':'NFC手機信用卡',
'北港朝天宮認同卡':'北港朝天宮認同卡',
'元大鈦金卡':'元大鈦金卡',
'COSTCO聯名卡':'COSTCO聯名卡',
'元大Life卡':'元大Life卡',
'中興保全聯名卡':'中興保全聯名卡',
'合作金庫世界卡':'合作金庫世界卡',
'學學認同卡':'學學認同卡',
'哆啦A夢卡':'哆啦A夢卡',
'歐付寶悠遊聯名卡':'歐付寶悠遊聯名卡',
'一卡通聯名晶緻卡':'一卡通聯名晶緻卡',
'幸福卡':'幸福卡',
'鈦金商旅卡':'鈦金商旅卡',
'中油職工福利認同卡':'中油職工福利認同卡',
'臺灣大學卡':'臺灣大學卡',
'悠遊聯名卡':'悠遊聯名卡',
'澳盛現金回饋卡':'澳盛現金回饋卡',
'中華郵政工會認同卡':'中華郵政工會認同卡',
'彰化六信聯名卡':'彰化六信聯名卡',
'玉山雙幣信用卡':'玉山雙幣信用卡',
'京城銀行聯名卡':'京城銀行聯名卡',
'頂好超市聯名卡':'頂好超市聯名卡',
'高雄醫學院認同卡':'高雄醫學院認同卡',
'美元雙幣信用卡':'美元雙幣信用卡',
'世界卡':'世界卡',
'聯邦銀行世界卡':'聯邦銀行世界卡',
'環球無限卡':'環球無限卡',
'高雄觀光福利聯名卡':'高雄觀光福利聯名卡',
'中國信託紅利卡':'中國信託紅利卡',
'元大鑽金icash聯名卡':'元大鑽金icash聯名卡',
'佐登妮絲聯名卡':'佐登妮絲聯名卡',
'故宮之友卡':'故宮之友卡',
'媚儷鈦金卡':'媚儷鈦金卡',
'台塑(悠遊)聯名卡':'台塑(悠遊)聯名卡',
'聯邦樂活一卡通聯名卡':'聯邦樂活一卡通聯名卡',
'三信商業銀行信用卡':'三信商業銀行信用卡',
'玉山eTag悠遊聯名卡':'玉山eTag悠遊聯名卡',
'分期便利卡':'分期便利卡',
'大潤發油樂聯名卡':'大潤發油樂聯名卡',
'玫瑰卡':'玫瑰卡',
'LOVE晶緻悠遊聯名卡－寵愛紅卡':'LOVE晶緻悠遊聯名卡－寵愛紅卡',
'元大台灣霹靂卡':'元大台灣霹靂卡',
'世界商務卡':'世界商務卡',
'超級現金回饋卡':'超級現金回饋卡',
'個人商旅卡':'個人商旅卡',
'聯邦旅遊卡':'聯邦旅遊卡',
'Gogoro聯名卡':'Gogoro聯名卡',
'利high卡':'利high卡',
'淡水信用合作社聯名卡':'淡水信用合作社聯名卡',
'ｉ網購生活卡':'ｉ網購生活卡',
'哆啦A夢悠遊聯名卡':'哆啦A夢悠遊聯名卡',
'花旗現金回饋(悠遊)卡':'花旗現金回饋(悠遊)卡',
'頂好Visa payWave聯名卡':'頂好Visa payWave聯名卡',
'享購鈦金卡':'享購鈦金卡',
'台灣黑熊認同卡':'台灣黑熊認同卡',
'酷比悠遊聯名卡':'酷比悠遊聯名卡',
'玉山晶緻悠遊卡':'玉山晶緻悠遊卡',
'商旅卡':'商旅卡',
'日盛ALL PASS卡':'日盛ALL PASS卡',
'商務白金卡':'商務白金卡',
'原子小金剛悠遊聯名卡':'原子小金剛悠遊聯名卡',
'藝術白金卡':'藝術白金卡',
'iCash愛現':'iCash愛現',
'日盛GOGO加油卡':'日盛GOGO加油卡',
'雙幣鈦金商旅卡':'雙幣鈦金商旅卡',
'台南三信聯名卡':'台南三信聯名卡',
'鹿港天后宮鈦金悠遊認同卡':'鹿港天后宮鈦金悠遊認同卡',
'新光銀行分期7卡':'新光銀行分期7卡',
'Hello Kitty分享聯名卡商務':'Hello Kitty分享聯名卡商務',
'iCash聯名卡':'iCash聯名卡',
'中國人壽聯名卡':'中國人壽聯名卡',
'Combo晶片卡':'Combo晶片卡',
'國泰人壽聯名卡':'國泰人壽聯名卡',
'永豐財富無限卡':'永豐財富無限卡',
'澎湖戀戀菊島認同卡':'澎湖戀戀菊島認同卡',
'銀色之愛鈦商卡':'銀色之愛鈦商卡',
'樂活卡(原澳盛)':'樂活卡(原澳盛)',
'愛Pass鈦金卡':'愛Pass鈦金卡',
'幸福御守卡／粉':'幸福御守卡／粉',
'大甲媽祖認同卡':'大甲媽祖認同卡',
'微風廣場 JSpeedy感應式聯名卡':'微風廣場 JSpeedy感應式聯名卡',
'e秒萬事通金融信用卡':'e秒萬事通金融信用卡',
'台灣企銀商務卡':'台灣企銀商務卡',
'國民旅遊卡':'國民旅遊卡',
'金采卡':'金采卡',
'燦坤生活聯名卡':'燦坤生活聯名卡',
'運籌理財信用卡':'運籌理財信用卡',
'台北市第五信用合作社聯名卡':'台北市第五信用合作社聯名卡',
'元大樂遊卡':'元大樂遊卡',
'鑑賞家白金卡':'鑑賞家白金卡',
'大立無限卡':'大立無限卡',
'輔仁大學認同卡':'輔仁大學認同卡',
'天仁茗茶聯名卡':'天仁茗茶聯名卡',
'SunnyCard':'SunnyCard',
'華歌爾鈦金悠遊聯名卡':'華歌爾鈦金悠遊聯名卡',
'Hello Kitty點數聯名卡商務':'Hello Kitty點數聯名卡商務',
'台新財富無限卡':'台新財富無限卡',
'卡娜赫拉的小動物icash聯名卡':'卡娜赫拉的小動物icash聯名卡',
'勞動保障信用卡':'勞動保障信用卡',
'醫師尊榮無限卡':'醫師尊榮無限卡',
'里仁為美福智卡':'里仁為美福智卡',
'元大商務鈦金卡':'元大商務鈦金卡',
'新竹市民認同卡':'新竹市民認同卡',
'逢甲人一卡通鈦金認同卡':'逢甲人一卡通鈦金認同卡',
'遠傳friDay聯名卡':'遠傳friDay聯名卡',
'麗寶悠遊聯名卡':'麗寶悠遊聯名卡',
'玉山銀行卡':'玉山銀行卡',
'元大鑽金卡':'元大鑽金卡',
'無限卡':'無限卡',
'Hello Kitty鑽金聯名卡商務':'Hello Kitty鑽金聯名卡商務',
'漢神百貨聯名卡':'漢神百貨聯名卡',
'富邦數位生活卡':'富邦數位生活卡',
'國璽聯名卡':'國璽聯名卡',
'HappyCash & HAPPY GO聯名卡':'HappyCash & HAPPY GO聯名卡',
'玉山教師鈦金卡':'玉山教師鈦金卡',
'FlyGo商務':'FlyGo商務',
'白金商旅卡':'白金商旅卡',
'錢進卡':'錢進卡',
'飲食男女聯名卡':'飲食男女聯名卡',
'NISSAN感心悠遊聯名卡':'NISSAN感心悠遊聯名卡',
'中華溫馨服務卡':'中華溫馨服務卡',
'樂活悠遊鈦金卡':'樂活悠遊鈦金卡',
'京華城聯名卡':'京華城聯名卡',
'台灣企銀信用卡':'台灣企銀信用卡',
'MUJI無印良品聯名卡':'MUJI無印良品聯名卡',
'台大EMBA基金會白金卡':'台大EMBA基金會白金卡',
'富邦尊御世界卡':'富邦尊御世界卡',
'美福聯名卡':'美福聯名卡',
'玉山統一時代悠遊聯名卡':'玉山統一時代悠遊聯名卡',
'永豐國民旅遊卡':'永豐國民旅遊卡',
'新光三越聯名卡':'新光三越聯名卡',
'士林高商校友認同卡':'士林高商校友認同卡',
'NewCentury無限卡':'NewCentury無限卡',
'中醫師無限卡':'中醫師無限卡',
'旅鑽個人商務信用卡':'旅鑽個人商務信用卡',
'元大鑽金一卡通聯名卡':'元大鑽金一卡通聯名卡',
'信望愛認同卡':'信望愛認同卡',
'鼎極卡':'鼎極卡',
'華銀白金卡':'華銀白金卡',
'SPORT卡商務':'SPORT卡商務',
'鼎鑽財富無限卡':'鼎鑽財富無限卡',
'嘉義三信聯名卡':'嘉義三信聯名卡',
'愛心卡':'愛心卡',
'農金一卡通聯名卡':'農金一卡通聯名卡',
'中華航空聯名卡':'中華航空聯名卡',
'兆豐悠遊聯名卡':'兆豐悠遊聯名卡',
'雙幣鈦金商旅卡':'雙幣鈦金商旅卡',
'幸福加值卡':'幸福加值卡',
'桐花認同卡':'桐花認同卡',
'新光三越悠遊聯名卡':'新光三越悠遊聯名卡',
'清華大學悠遊認同卡':'清華大學悠遊認同卡',
'VOLVO汽車聯名卡':'VOLVO汽車聯名卡',
'亞洲無限卡':'亞洲無限卡',
'聯邦農金聯名卡':'聯邦農金聯名卡',
'亞洲微創手術中心認同卡':'亞洲微創手術中心認同卡',
'大買家油樂聯名卡':'大買家油樂聯名卡',
'台灣大哥大悠遊聯名卡':'台灣大哥大悠遊聯名卡',
'ETC聯名卡':'ETC聯名卡',
'原子小金剛卡':'原子小金剛卡',
'享利白金卡':'享利白金卡',
'Bankee卡':'Bankee卡',
'兆豐世界卡':'兆豐世界卡',
'耐斯廣場NP聯名卡':'耐斯廣場NP聯名卡',
'萬事通悠遊金融信用卡':'萬事通悠遊金融信用卡',
'富多卡':'富多卡',
'ｉ網購生活卡':'ｉ網購生活卡',
'台塑(一卡通)聯名卡商務':'台塑(一卡通)聯名卡商務',
'哆啦A夢悠遊晶緻卡':'哆啦A夢悠遊晶緻卡',
'大甲媽祖悠遊認同卡':'大甲媽祖悠遊認同卡',
'玉山雙幣信用卡':'玉山雙幣信用卡',
'獅子之友認同卡':'獅子之友認同卡',
'太陽卡':'太陽卡',
'農金悠遊聯名卡':'農金悠遊聯名卡',
'理財型白金卡':'理財型白金卡',
'東海大學悠遊認同卡':'東海大學悠遊認同卡',
'導盲犬認同卡':'導盲犬認同卡',
'桃園市市民卡聯名信用卡':'桃園市市民卡聯名信用卡',
'比漾聯名卡':'比漾聯名卡',
'Global Mall聯名卡':'Global Mall聯名卡',
'KOKO COMBO icash':'KOKO COMBO icash',
'夢時代icash 聯名卡':'夢時代icash 聯名卡',
'陽信鈦金卡':'陽信鈦金卡',
'VOGUE聯名卡':'VOGUE聯名卡',
'NFC手機信用卡':'NFC手機信用卡',
'現金回饋悠遊鈦金卡':'現金回饋悠遊鈦金卡',
'Display金融信用卡':'Display金融信用卡',
'信義房屋聯名卡':'信義房屋聯名卡',
'現金回饋悠遊鈦金Combo卡':'現金回饋悠遊鈦金Combo卡',
'微風廣場Visa Pay Wave感應式聯名卡':'微風廣場Visa Pay Wave感應式聯名卡',
'陽信櫻花白金卡':'陽信櫻花白金卡',
'微風聯名卡':'微風聯名卡',
'行動鈦金卡':'行動鈦金卡',
'玉山統一時代icash聯名卡':'玉山統一時代icash聯名卡',
'NFC寰宇鈦商手機信用卡':'NFC寰宇鈦商手機信用卡',
'家扶公益認同卡':'家扶公益認同卡',
'花旗HAPPY GO聯名卡':'花旗HAPPY GO聯名卡',
'美樂家生活卡':'美樂家生活卡',
'兆豐無限卡':'兆豐無限卡',
'手機信用卡':'手機信用卡',
'ESPRIT聯名卡':'ESPRIT聯名卡',
'樂天信用卡':'樂天信用卡',
'晶片Combo card':'晶片Combo card',
'一卡通聯名鈦金卡':'一卡通聯名鈦金卡',
'棉花田悠遊聯名卡':'棉花田悠遊聯名卡',
'新世代信用卡':'新世代信用卡',
'中國醫藥大學暨附設醫院晶緻認同卡':'中國醫藥大學暨附設醫院晶緻認同卡',
'Combo Life卡':'Combo Life卡',
'遠東HAPPY GO頂級信用卡':'遠東HAPPY GO頂級信用卡',
'南紡夢時代聯名卡':'南紡夢時代聯名卡',
'永豐Me Display Card':'永豐Me Display Card',
'分享卡':'分享卡',
'中華電信感恩卡':'中華電信感恩卡',
'微風悠遊聯名卡':'微風悠遊聯名卡',
'經典鈦金卡':'經典鈦金卡',
'I-FIRST生活卡':'I-FIRST生活卡',
'雙幣鈦金商旅卡':'雙幣鈦金商旅卡',
'C’est Moi旅遊悠遊卡':'C’est Moi旅遊悠遊卡',
'中興大學認同卡':'中興大學認同卡',
'大葉高島屋聯名卡':'大葉高島屋聯名卡',
'TheShoppingCard分期卡':'TheShoppingCard分期卡',
'NewCentury世界卡':'NewCentury世界卡',
'新光銀行世界卡':'新光銀行世界卡',
'中華電信商務鈦金感恩卡':'中華電信商務鈦金感恩卡',
'澎湖iPASS卡':'澎湖iPASS卡',
'LOVE晶緻悠遊聯名卡－酷愛黑卡':'LOVE晶緻悠遊聯名卡－酷愛黑卡',
'合庫雙幣信用卡':'合庫雙幣信用卡',
'台塑(悠遊)聯名卡商務':'台塑(悠遊)聯名卡商務',
'富貴無限卡':'富貴無限卡',
'美安悠遊聯名卡':'美安悠遊聯名卡',
'山隆優油卡':'山隆優油卡',
'金采鈦商卡':'金采鈦商卡',
'順發多謝聯名卡':'順發多謝聯名卡',
'MAZDA天空卡':'MAZDA天空卡',
'i-Fun愛玩樂卡':'i-Fun愛玩樂卡',
'凱基無限卡':'凱基無限卡',
'板信銀行聯名卡':'板信銀行聯名卡',
'葡眾聯名卡':'葡眾聯名卡',
'幸運PLUS鈦金卡':'幸運PLUS鈦金卡',
'大紀元認同卡':'大紀元認同卡',
'富邦財神手機卡':'富邦財神手機卡',
'得利COMBO晶片卡':'得利COMBO晶片卡',
'白沙屯拱天宮媽祖認同卡':'白沙屯拱天宮媽祖認同卡',
'HappyCash & HAPPY GO聯名卡':'HappyCash & HAPPY GO聯名卡',
'民間版國民旅遊卡':'民間版國民旅遊卡',
'鈦豐卡':'鈦豐卡',
'玩全南投認同卡':'玩全南投認同卡',
'宜蘭大學悠遊認同卡':'宜蘭大學悠遊認同卡',
'聯邦國民旅遊卡':'聯邦國民旅遊卡',
'Sweet Home白金卡':'Sweet Home白金卡',
'ｉ網購生活卡':'ｉ網購生活卡',
}

In [19]:
#article_social_seg=df_article_social['seg']
#article_web_seg=df_article_web['seg']
#article_news_seg=df_article_news['seg']
#bank
#news
n_soc=len(article_social_seg)
bank_tag_soc=[]
for article in range(n_soc):
    article_bank=set()
    for word in article_social_seg[article]:
            if word in bank:
                article_bank.add(bank[word])
    bank_tag_soc.append(list(article_bank))
print('soc')
print(bank_tag_soc[:10])

#web
n_web=len(article_web_seg)
bank_tag_web=[]
for article in range(n_web):
    article_bank=set()
    for word in article_web_seg[article]:
            if word in bank:
                article_bank.add(bank[word])
    bank_tag_web.append(list(article_bank))
print('web')
print(bank_tag_web[:10])


#social
n_news=len(article_news_seg)
bank_tag_news=[]
for article in range(n_news):
    article_bank=set()
    for word in article_news_seg[article]:
            if word in bank:
                article_bank.add(bank[word])
    bank_tag_news.append(list(article_bank))
print('news')
print(bank_tag_news[:10])

soc
[[], [], ['台新'], ['永豐', '臺灣銀行'], [], ['玉山', '富邦', '台新', '元大'], ['台新'], ['永豐'], ['Richart'], []]
web
[['玉山'], ['國泰'], ['星展', '華南', '永豐', 'Richart', '上海商銀', '匯豐', '中信', '台新'], ['永豐', '上海商銀', '中信', '聯邦', '華南', '一銀', '匯豐', '國泰', '兆豐', '元大', '陽信', '台新', '星展', '日盛', '富邦', '澳盛', '凱基', '彰化', '玉山', '花旗', '新光'], [], [], [], ['聯邦', '遠東商銀', '兆豐', '富邦', '台新', '玉山', '台中銀行', '一銀', '永豐', '國泰', '凱基', '元大', '合作金庫', '中信', '花旗', '新光'], ['永豐', '中信', '聯邦', '華南', '花旗', '國泰', '渣打', '陽信', '元大', '兆豐', '台新', '台中銀行', '日盛', '富邦', '凱基', '彰化', '玉山', '合作金庫', '新光'], ['上海商銀', '星展', '花旗', '兆豐', '富邦', '玉山', 'KOKO', '永豐', 'Richart', '澳盛', '國泰', '渣打', '元大', '匯豐', '中信', '台新']]
news
[['聯邦', 'KOKO', '富邦', '永豐', '國泰', '渣打', '玉山', '中信', '台新'], [], ['玉山', '台新'], ['高雄銀行', '永豐', '中信', '聯邦', '華南', '一銀', '華泰', '合作金庫', '國泰', '兆豐', '陽信', '元大', '渣打', '台新', '台中銀行', '遠東商銀', '星展', '日盛', '臺灣企銀', '土地', '富邦', '凱基', '玉山', '花旗', '新光'], ['玉山', '中信', '台新', '兆豐'], ['聯邦', '富邦', '一銀', '永豐', '國泰', '玉山', '中信', '台新'], ['聯邦', '華南', '富邦', '元大', '國泰'

In [20]:
#card
#soc

n_soc=len(article_social_seg)
card_tag_soc=[]
for article in range(n_soc):
    article_card=set()
    for word in article_social_seg[article]:
            if word in card:
                article_card.add(card[word])
    card_tag_soc.append(list(article_card))
print('soc')
print(card_tag_soc[:10])

#web
n_web=len(article_web_seg)
card_tag_web=[]
for article in range(n_web):
    article_card=set()
    for word in article_web_seg[article]:
            if word in card:
                article_card.add(card[word])
    card_tag_web.append(list(article_card))
print('web')
print(card_tag_web[:10])


#news
n_news=len(article_news_seg)
card_tag_news=[]
for article in range(n_news):
    article_card=set()
    for word in article_news_seg[article]:
            if word in card:
                article_card.add(card[word])
    card_tag_news.append(list(article_card))
print('news')
print(card_tag_news[:10])

soc
[[], [], ['街口聯名卡'], ['台灣銀行', 'dawho'], ['dawho'], [], ['GoGo', '悠遊聯名卡'], ['dawho'], ['LINEPay'], ['美國運通', '小小兵信用卡']]
web
[['Pi'], [], ['小小兵信用卡', 'Flygo', 'dawho', '炫晶御璽卡'], ['世界卡', '無限卡'], ['LINEPay'], [], [], ['悠遊聯名卡', 'Bankee卡', '樂天信用卡', 'LINEPay', '玉山銀行卡'], ['鼎極卡', '世界卡', '中醫師無限卡', '尊御世界卡', '商務白金卡', '醫師尊榮無限卡', '無限卡', '大立無限卡', '美福聯名卡', '經典鈦金卡', '建築師無限卡', '學學認同卡', '國民旅遊卡', '太陽卡', '鈦金商務卡', '玫瑰卡', '富貴無限卡', '個人商旅卡', '美國運通', '商務卡'], ['Flygo', '簡單卡', 'KOKO', 'e秒刷鈦金卡', 'dawho', '鑽金', 'GoGo', '現金回饋御璽卡', 'Pi', 'icash聯名卡', 'LINEPay', '炫晶御璽卡', 'Ubear']]
news
[['小小兵信用卡', '賴點卡', 'KOKO', 'dawho', '街口聯名卡', '現金回饋御璽卡', 'Pi', 'icash聯名卡', 'LINEPay', 'Only'], [], ['Pi'], ['LINEPay'], ['Pi', '街口聯名卡', 'LINEPay', '無限卡'], [], ['賴點卡', '蝦皮購物聯名卡', 'LINEPay', 'My樂現金回饋卡', 'Only'], ['SPORT卡', 'Only'], ['LINEPay'], []]


In [23]:
#behavior      
#soc

n_soc=len(article_social_seg)
behavior_tag_soc=[]
for article in range(n_soc):
    article_behavior=set()
    for word in article_social_seg[article]:
            if word in behavior:
                article_behavior.add(behavior[word])
    behavior_tag_soc.append(list(article_behavior))
print('soc')
print(behavior_tag_soc[:10])

#web
n_web=len(article_web_seg)
behavior_tag_web=[]
for article in range(n_web):
    article_behavior=set()
    for word in article_web_seg[article]:
            if word in behavior:
                article_behavior.add(behavior[word])
    behavior_tag_web.append(list(article_behavior))
print('web')
print(behavior_tag_web[:10])


#news
n_news=len(article_news_seg)
behavior_tag_news=[]
for article in range(n_news):
    article_behavior=set()
    for word in article_news_seg[article]:
            if word in behavior:
                article_behavior.add(behavior[word])
    behavior_tag_news.append(list(article_behavior))
print('news')
print(behavior_tag_news[:10])

soc
[[], [], ['行動支付'], ['旅遊'], [], [], [], [], [], ['量販店', '藥妝']]
web
[['國外', '超商', '行動支付', '旅遊'], ['旅遊'], ['國外', '旅遊', '網購', '航空哩程'], ['國外', '旅遊'], [], ['旅遊'], ['紅利', '保險'], ['行動支付', '加油'], ['國外', '旅遊'], ['超商', '電影', '加油', '保險', '國外', '旅遊', '網購', '航空哩程']]
news
[['超商', '行動支付', '旅遊', '電影', '網購'], ['網購'], ['網購', '保險'], ['行動支付'], ['國外', '超商', '網購', '行動支付'], ['超市', '超商', '行動支付', '量販店'], ['國外', '旅遊', '行動支付'], ['國外', '運動'], [], ['超市', '行動支付']]


In [24]:
#合併dataframe
#soc
d_soc={'bank_tag':bank_tag_soc,'card_tag':card_tag_soc,'behavior_tag':behavior_tag_soc}
df_tag_soc=pd.DataFrame(data=d_soc)


#web
d_web={'bank_tag':bank_tag_web,'card_tag':card_tag_web,'behavior_tag':behavior_tag_web}
df_tag_web=pd.DataFrame(data=d_web)


#news
d_news={'bank_tag':bank_tag_news,'card_tag':card_tag_news,'behavior_tag':behavior_tag_news}
df_tag_news=pd.DataFrame(data=d_news)


In [25]:
#df_tag_soc[:10]
#df_tag_web[:10]
df_tag_news[:10]


,bank_tag,card_tag,behavior_tag
0,"[聯邦, KOKO, 富邦, 永豐, 國泰, 渣打, 玉山, 中信, 台新]","[小小兵信用卡, 賴點卡, KOKO, dawho, 街口聯名卡, 現金回饋御璽卡, Pi,...","[超商, 行動支付, 旅遊, 電影, 網購]"
1,[],[],[網購]
2,"[玉山, 台新]",[Pi],"[網購, 保險]"
3,"[高雄銀行, 永豐, 中信, 聯邦, 華南, 一銀, 華泰, 合作金庫, 國泰, 兆豐, 陽...",[LINEPay],[行動支付]
4,"[玉山, 中信, 台新, 兆豐]","[Pi, 街口聯名卡, LINEPay, 無限卡]","[國外, 超商, 網購, 行動支付]"
5,"[聯邦, 富邦, 一銀, 永豐, 國泰, 玉山, 中信, 台新]",[],"[超市, 超商, 行動支付, 量販店]"
6,"[聯邦, 華南, 富邦, 元大, 國泰, 渣打, 玉山, 花旗, 中信, 台新]","[賴點卡, 蝦皮購物聯名卡, LINEPay, My樂現金回饋卡, Only]","[國外, 旅遊, 行動支付]"
7,"[玉山, 永豐]","[SPORT卡, Only]","[國外, 運動]"
8,"[Richart, 富邦, 中信, 台新]",[LINEPay],[]
9,"[聯邦, 華泰, 富邦, 一銀, 永豐, 國泰, 玉山, 中信, 台新]",[],"[超市, 行動支付]"


In [ ]:
db_ETL.list_collection_names()

In [39]:
data_tag_news = json.loads(df_tag_news.T.to_json()).values()

In [42]:
data_tag_soc = json.loads(df_tag_soc.T.to_json()).values()
data_tag_web = json.loads(df_tag_web.T.to_json()).values()

In [41]:
#丟mongodb

db_ETL.article_tag_news.insert_many(data_tag_news)

In [44]:
db_ETL.article_tag_soc.insert_many(data_tag_soc)
db_ETL.article_tag_web.insert_many(data_tag_web)

In [45]:
#存本地
df_tag_soc.to_json('./article_social_tag.json',orient='index',force_ascii=False)
df_tag_web.to_json('./article_web_tag.json',orient='index',force_ascii=False)
df_tag_news.to_json('./article_news_tag.json',orient='index',force_ascii=False)

In [26]:
#計算card behavior出現次數矩陣
all_bank=set([bank[b] for b in bank])
all_card=set([card[c] for c in card])
all_behavior=set([behavior[b] for b in behavior])


#建立df並填0
df_card_behavoir=pd.DataFrame(index=all_card,columns=all_behavior)
for i in df_card_behavoir.columns:
    df_card_behavoir[i]=0
df_card_behavoir

,保險,紅利,電影,超商,量販店,藥妝,超市,運動,行動支付,百貨,加油,國外,旅遊,網購,航空哩程
勞動保障信用卡,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
e秒刷鈦金卡,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
永豐國民旅遊卡,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
鈦金商旅卡,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
元大鑽金卡,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
晶片Combo card,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
台塑(悠遊)聯名卡商務,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
比漾聯名卡,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
士林高商校友認同卡,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
元大鈦金卡,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
df_card_behavoir_soc=df_card_behavoir
df_card_behavoir_web=df_card_behavoir
df_card_behavoir_news=df_card_behavoir

In [28]:
#計算
#soc

ntag_soc=len(df_tag_soc)
#填入df之value
for i in  range(ntag_soc):
    for c_tag in df_tag_soc['card_tag'][i]:
        for b_tag in df_tag_soc['behavior_tag'][i]:
            df_card_behavoir_soc[b_tag].loc[c_tag]+=1
df_card_behavoir_soc

,保險,紅利,電影,超商,量販店,藥妝,超市,運動,行動支付,百貨,加油,國外,旅遊,網購,航空哩程
勞動保障信用卡,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
e秒刷鈦金卡,45,14,60,63,67,8,21,8,89,44,64,99,106,187,17
永豐國民旅遊卡,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
鈦金商旅卡,21,8,27,22,23,4,13,5,26,29,30,37,40,32,12
元大鑽金卡,17,3,14,17,15,0,4,1,16,7,22,65,75,33,3
晶片Combo card,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
台塑(悠遊)聯名卡商務,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
比漾聯名卡,0,0,1,0,3,0,0,0,0,0,0,0,0,0,0
士林高商校友認同卡,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
元大鈦金卡,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [29]:
#web

ntag_web=len(df_tag_web)
#填入df之value
for i in  range(ntag_web):
    for c_tag in df_tag_web['card_tag'][i]:
        for b_tag in df_tag_web['behavior_tag'][i]:
            df_card_behavoir_web[b_tag].loc[c_tag]+=1
df_card_behavoir_web

,保險,紅利,電影,超商,量販店,藥妝,超市,運動,行動支付,百貨,加油,國外,旅遊,網購,航空哩程
勞動保障信用卡,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
e秒刷鈦金卡,63,24,88,88,76,13,31,9,119,46,83,134,131,235,30
永豐國民旅遊卡,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
鈦金商旅卡,23,8,28,23,23,4,13,5,27,29,31,39,41,33,12
元大鑽金卡,25,7,24,22,20,2,9,1,24,9,32,96,98,43,9
晶片Combo card,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
台塑(悠遊)聯名卡商務,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
比漾聯名卡,0,0,1,0,3,0,0,0,0,0,0,0,0,0,0
士林高商校友認同卡,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
元大鈦金卡,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [30]:
#news

ntag_news=len(df_tag_news)
#填入df之value
for i in  range(ntag_news):
    for c_tag in df_tag_news['card_tag'][i]:
        for b_tag in df_tag_news['behavior_tag'][i]:
            df_card_behavoir_news[b_tag].loc[c_tag]+=1
df_card_behavoir_news

,保險,紅利,電影,超商,量販店,藥妝,超市,運動,行動支付,百貨,加油,國外,旅遊,網購,航空哩程
勞動保障信用卡,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
e秒刷鈦金卡,65,25,88,93,76,14,32,10,123,46,84,141,138,247,30
永豐國民旅遊卡,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
鈦金商旅卡,23,8,28,23,23,4,13,5,27,29,31,40,43,34,12
元大鑽金卡,25,8,25,23,21,2,9,1,25,10,33,102,104,45,10
晶片Combo card,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
台塑(悠遊)聯名卡商務,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
比漾聯名卡,0,0,1,0,3,0,0,0,0,0,0,0,0,0,0
士林高商校友認同卡,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
元大鈦金卡,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_tag_soc[:10]
df_tag_web[:10]
df_tag_news[:10]

In [32]:
combine=[df_tag_soc,df_tag_web,df_tag_news]

In [34]:
df_tag_all=pd.concat(combine)

In [35]:
len(df_tag_all)+1

20547

In [ ]:
df_tag_all[:10]

In [ ]:
df_card_behavoir_all=df_card_behavoir

In [38]:
#all
ntag_all=len(df_tag_all)
#填入df之value
for i in  range(ntag_all):
    for c_tag in df_tag_all['card_tag'][i]:
        for b_tag in df_tag_all['behavior_tag'][i]:
            df_card_behavoir_all[b_tag].loc[c_tag]+=1
df_card_behavoir_all[:2]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


KeyboardInterrupt: 

In [31]:
#export
df_card_behavoir_news.to_excel(r'./card_behavoir_news_matrix.xlsx','w',encoding='utf-16')
df_card_behavoir_web.to_excel(r'./card_behavoir_web_matrix.xlsx','w',encoding='utf-16')
df_card_behavoir_soc.to_excel(r'./card_behavior_soc_matrix.xlsx','w',encoding='utf-16')
df_card_behavoir_all.to_excel(r'./card_behavior_all_matrix.xlsx','w',encoding='utf-16')